In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "bitcoindata-457720-f464ff7656ea.json"

In [2]:
from google.cloud import bigquery
import pandas as pd

In [3]:
client = bigquery.Client()

In [4]:
query = """
SELECT
  t.hash,
  t.size,
  ARRAY_LENGTH(t.inputs) AS vin_sz,
  ARRAY_LENGTH(t.outputs) AS vout_sz,
  (SELECT SUM(i.value) FROM UNNEST(t.inputs) i) AS input_value,
  (SELECT SUM(o.value) FROM UNNEST(t.outputs) o) AS output_value,
  (SELECT SUM(i.value) FROM UNNEST(t.inputs) i) -
  (SELECT SUM(o.value) FROM UNNEST(t.outputs) o) AS fee,
  t.block_timestamp
FROM `bigquery-public-data.crypto_bitcoin.transactions` t TABLESAMPLE SYSTEM (1 PERCENT)
WHERE t.block_timestamp IS NOT NULL
ORDER BY t.block_timestamp DESC
LIMIT 100000
"""

In [5]:
df = client.query(query).to_dataframe()

/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [6]:
df.head()

,hash,size,vin_sz,vout_sz,input_value,output_value,fee,block_timestamp
0,36212c4d55815e0a04733a0500926ef38876920404f041...,194,1,1,417021.000000000,416569.000000000,452.000000000,2025-04-18 23:47:46+00:00
1,11ec29c3ac884fb4c1f12fab1b9c455205c70069295130...,205,1,2,14725.000000000,14263.000000000,462.000000000,2025-04-18 23:47:46+00:00
2,89d8d27e4183ebbb4aaf0ba1f1d708d610fbe82d79a8ec...,229,1,2,795.000000000,330.000000000,465.000000000,2025-04-18 23:47:46+00:00
3,5a59d71ccc66cafa3af24e2f4a6a2eee5bfcab5183de58...,194,1,1,249990.000000000,249651.000000000,339.000000000,2025-04-18 23:47:46+00:00
4,ee23fe345a6820721dbd0e7e7c7e7825a2920972787085...,224,1,2,1493283.000000000,1492573.000000000,710.000000000,2025-04-18 23:47:46+00:00
